In [52]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException
import re
from time import sleep as time_sleep
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from pathlib import Path
import lxml
import pandas as pd
import numpy as np

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[0]
print(zip_file)

[WindowsPath('data/2023-04-25_00-30-lenta.zip')]
data\2023-04-25_00-30-lenta.zip


In [34]:
pages_list = []
with ZipFile(zip_file, 'r') as zf:
    for item in zf.filelist:
        with zf.open(item.filename) as file:
            pages_list.append(file.read())

In [36]:
pages_list = pages_list[300:400]

In [13]:
def _test_01(page: str):
    soup = BS(page, 'lxml')
    table = soup.find('div', class_='catalog-grid__grid catalog-grid').find_all('a', class_='lui-card lui-sku-product-card sku-card-small-container catalog-grid-sku-product-card')
    return len(table)

test = []
for page in pages_list:
    test.append(_test_01(page))
    
df = pd.Series(test)
df.value_counts()

24    99
23     1
dtype: int64

In [11]:
"""
Отлично, количество плиток не превышает 24 плитки.
Также нужно определиться что бужем загружать из карточек:
    Название,
    Категория,
    Производитель,
    Цена по карте,
    Обычная цена,
    Скидка, если есть
    Количество...
""";

In [17]:
def _test_02(page: str):
    out_list = []
    soup = BS(page, 'lxml')
    table = soup.find('div', class_='catalog-grid__grid catalog-grid').find_all('a', class_='lui-card lui-sku-product-card sku-card-small-container catalog-grid-sku-product-card')
    for item in table:
        for it in item.find_next_siblings():
            out_list.append(it.get('class'))
    
    return out_list

test = []
for page in pages_list[:1]:
    test.append(_test_02(page))

test = sum(test, [])
df = pd.Series(test)
df

0      [lui-card, lui-sku-product-card, sku-card-smal...
1      [lui-card, lui-sku-product-card, sku-card-smal...
2      [lui-card, lui-sku-product-card, sku-card-smal...
3      [lui-card, lui-sku-product-card, sku-card-smal...
4      [lui-card, lui-sku-product-card, sku-card-smal...
                             ...                        
271    [lui-card, lui-sku-product-card, sku-card-smal...
272    [lui-card, lui-sku-product-card, sku-card-smal...
273    [lui-card, lui-sku-product-card, sku-card-smal...
274    [lui-card, lui-sku-product-card, sku-card-smal...
275    [lui-card, lui-sku-product-card, sku-card-smal...
Length: 276, dtype: object

In [30]:
def _test_03(page):
    out_list = []
    soup = BS(page, 'lxml')
    table = soup.find('div', class_='catalog-grid__grid catalog-grid')
    cards = table.find_all('a', class_='lui-card lui-sku-product-card sku-card-small-container catalog-grid-sku-product-card')
    for item in cards:
        element = item.find('div')
        out_list.append(element.get('class'))
        outers_elements = element.find_next_siblings()
        for elem in outers_elements:
            out_list.append(elem.get('class'))
    return out_list

In [35]:

result = []
for page in pages_list:
    result.append(_test_03(page))
result = sum(result, [])

df = pd.Series(result)
df.value_counts()

[lui-sku-product-card-preview]                                            30888
[lui-sku-product-card-text, lui-sku-product-card-text--view-primary]      30888
[lui-sku-product-card-text, lui-sku-product-card-text--view-secondary]    30888
[lui-sku-product-card-price]                                              30888
[sku-client-card-small-control, lui-sku-product-card-control]             30888
dtype: int64

In [37]:
"""
    Проанализировав весь сайт, выяснили, что в карточке товара есть пять классов с информацией.
    lui-sku-product-card-preview        Здесь можно взять количество звезд (нафиг нада конечно)
                                        Так же указание на скидку
    lui-sku-product-card-text, lui-sku-product-card-text--view-primary      Здесь назвиние продукта
    lui-sku-product-card-text, lui-sku-product-card-text--view-secondary    Здесь страна производитель
    lui-sku-product-card-price      Здесь текущая цена и по карте
    Пока все
""";

In [93]:
page = pages_list[0]

soup = BS(page, 'lxml')
table = soup.find('div', class_='catalog-grid__grid catalog-grid')
cards = table.find_all('a', class_='lui-card lui-sku-product-card sku-card-small-container catalog-grid-sku-product-card')
# card = table.find('a', class_='lui-card lui-sku-product-card sku-card-small-container catalog-grid-sku-product-card')
def _get_products(card: BS) -> pd.DataFrame:
    # Скидка если есть
    sale = card.find('div', class_='lui-sku-product-card-preview').find('span', class_='lui-badge lui-badge_discount') # Нужно проверить на большом количестве
    try:
        sale = sale.text
    except AttributeError as ex:
        sale = np.nan
    # Название продукта
    name = card.find('div', class_='lui-sku-product-card-text lui-sku-product-card-text--view-primary').get_text(strip=True)

    # Страна производитель
    made_in = card.find('div', class_='lui-sku-product-card-text lui-sku-product-card-text--view-secondary').get_text(strip=True)

    # Цены
    price = card.find('div', class_='lui-sku-product-card-price').find('div', class_='lui-priceText lui-priceText--view_secondary').stripped_strings
    try:
        price = float('.'.join(list(price)[:2]))
    except Exception as ex:
        print(ex, 'price', price)
        price = np.nan

    price_on_cart = card.find('div', class_='lui-sku-product-card-price').find('div', class_='lui-priceText lui-priceText--view_regular').stripped_strings
    try:
        price_on_cart = float('.'.join(list(price_on_cart)[:2]))
    except Exception as ex:
        print(ex, 'price_on_cart', price_on_cart)
        price_on_cart = np.nan
    df = pd.DataFrame([[name, made_in, sale, price, price_on_cart]], columns=['Название', 'Страна', 'Скидка', 'Цена', 'Цена по карте'])
    return df

test = _get_products(table)
display(test)

,Название,Страна,Скидка,Цена,Цена по карте
0,"Смесь для супа ТРАПЕЗА На первое Рассольник, 130г","Россия, 130 г",NaN,119.99,126.39


In [97]:
category = soup.find('div', class_='catalog-view__header').find('h1', class_='catalog-view__title').get_text(strip=True)
(category)

'Бакалея'